In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
temp_dir = 'gdrive/MyDrive/CV_CW_temp/ResNet50'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# importing of modules for CNN 
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import utils
from tensorflow.keras.optimizers import SGD, RMSprop, Adam

# importing of service libraries
import numpy as np
import matplotlib.pyplot as plt

print('Libraries imported.')

Libraries imported.


In [3]:
# seed for reproducibility
np.random.seed(1)  

# network and training parameters
N_EPOCH = 20 # later use 20 for better results
BATCH_SIZE = 128
VERBOSE = 1
OPTIMIZER = Adam()
VALIDATION_SPLIT=0.2

IMG_ROWS, IMG_COLS = 32, 32 # input dimensions of images
N_CLASSES = 10  # number of outputs = number of classes
INPUT_SHAPE = (IMG_ROWS, IMG_COLS, 3) # 1st Dimension, 2nd Dimension, RGB channel

print('Main variables initialised.')

Main variables initialised.


In [4]:
# data: shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
# K.set_image_dim_ordering("th")

# consider them as float and normalize
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255 
X_test /= 255  

# we need a 60K x [3 x 32 x 32] shape as input to the CONVNET
# X_train = X_train[:, :, :, np.newaxis]
# X_test = X_test[:, :, :, np.newaxis]

# convert class vectors to binary class matrices
y_train = utils.to_categorical(y_train, N_CLASSES)
y_test = utils.to_categorical(y_test, N_CLASSES)

print(X_train.shape, 'X train shape')
print(X_test.shape, 'X test shape')
print(y_train.shape, 'y train shape')
print(y_test.shape, 'y test shape')

(50000, 32, 32, 3) X train shape
(10000, 32, 32, 3) X test shape
(50000, 10) y train shape
(10000, 10) y test shape


In [5]:
from tensorflow.keras.applications import resnet50

#define the convnet 
class Net1:
	@staticmethod
	def build(input_shape, classes):
		model = resnet50.ResNet50(classes=classes, weights=None, input_shape=input_shape)

		return model

print('Net class defined.')

Net class defined.


In [6]:
# initialize the optimizer and compile the model
model = Net1.build(input_shape=INPUT_SHAPE, classes=N_CLASSES)

model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER, metrics=["accuracy"])

model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                           

In [8]:
# additional checkpoint load code and temp directory definition
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

checkpoint_path = temp_dir+"/net.ckpt"

# definition of checkpoint parameters file
checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_best_only=True)

In [ ]:
# training/fitting of the Net model
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=N_EPOCH, verbose=VERBOSE, validation_split=VALIDATION_SPLIT, callbacks=[checkpoint])

Epoch 1/20
313/313 [==============================] - ETA: 0s - loss: 1.9487 - accuracy: 0.3541
Epoch 1: val_loss improved from inf to 3.17833, saving model to gdrive/MyDrive/CV_CW_temp/ResNet50/net.ckpt
INFO:tensorflow:Assets written to: gdrive/MyDrive/CV_CW_temp/ResNet50/net.ckpt/assets
313/313 [==============================] - 2514s 8s/step - loss: 1.9487 - accuracy: 0.3541 - val_loss: 3.1783 - val_accuracy: 0.1547
Epoch 2/20
313/313 [==============================] - ETA: 0s - loss: 1.4923 - accuracy: 0.4844
Epoch 2: val_loss improved from 3.17833 to 1.45243, saving model to gdrive/MyDrive/CV_CW_temp/ResNet50/net.ckpt
INFO:tensorflow:Assets written to: gdrive/MyDrive/CV_CW_temp/ResNet50/net.ckpt/assets
313/313 [==============================] - 2517s 8s/step - loss: 1.4923 - accuracy: 0.4844 - val_loss: 1.4524 - val_accuracy: 0.4766
Epoch 3/20
313/313 [==============================] - ETA: 0s - loss: 1.3781 - accuracy: 0.5310
Epoch 3: val_loss improved from 1.45243 to 1.37694, sa

In [ ]:
new_model = load_model(checkpoint_path)
remain_n_epoch = N_EPOCH - history.params['epochs']
if remain_n_epoch == 0:
  print("Train finished.")
else:
  new_history = new_model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=remain_n_epoch, verbose=VERBOSE, validation_split=VALIDATION_SPLIT, callbacks=[checkpoint])

In [ ]:
score = model.evaluate(X_test, y_test, verbose=VERBOSE)
print("\nTest score/loss:", score[0])
print('Test accuracy:', score[1])

# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Save Model Configures

In [ ]:
from tensorflow.keras.models import model_from_json

#Creation of of the model configuration in jsom format
model_json = model.to_json()

#save model in json format into a file
open(temp_dir+'Net_architecture.json', 'w').write(model_json)

print('Model definition jsom file saved.')

#model reconstruction from json format
model = model_from_json(model_json)


# Now let's compile, summarise and then train/fit the model.
model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER, metrics=["accuracy"])

model.summary()

## Save Model Weights

In [ ]:
#This saves the weights at the last epoch of the previous training session
model.save(temp_dir+'net.weights')
print ('Weights saved for final epoch ', N_EPOCH)
print ('Check that there is a weight file in your folder')

In [ ]:
from tensorflow.keras.models import load_model 

#This loads the weight file
model = load_model(temp_dir+'net.weights')
print ('Loading of the saved weights and test using these trained weights')

# let's check that the weighst loaded in this examples have kept the trained state of the network.
score = model.evaluate(X_test, y_test, verbose=VERBOSE)
print("\nTest score/loss:", score[0])
print('Test accuracy:', score[1])

## Checkpoint Saving

In [ ]:
# # additional checkpoint load code and temp directory definition
# from tensorflow.keras.callbacks import ModelCheckpoint
# from tensorflow.keras.models import load_model 

# # let's reduce the number of epoch to 3, for a faster test of the checkpoint utility
# N_EPOCH = 3


# # definition of checkpoint parameters file
# checkpoint = ModelCheckpoint(temp_dir+"1-{epoch:02d}.weights")

# print ('Training with checkpoint at each epoch. Check that at the end of each epoch, there is a weight file in the tmp folder')
# model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=N_EPOCH, validation_split=0.5, callbacks=[checkpoint])

# score = model.evaluate(X_test, y_test, verbose=VERBOSE)
# print("\nTest score/loss:", score[0])
# print('Test accuracy:', score[1])

# # Let's load the weights of the 2nd epoch, and check the scores 
# model = load_model(temp_dir+'1-02.weights')
# print ('Weights loaded for epoch 2 .')
# print ('New test using these trained weights.')

# # let's check that the weights loaded in this examples have kept the trained state of the network.
# score = model.evaluate(X_test, y_test, verbose=VERBOSE)
# print("\nTest score/loss:", score[0])
# print('Test accuracy:', score[1])